In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import os 
os.chdir('/content/drive/MyDrive/vietnamesesentiment ')


In [5]:
!pip install transformers

     |████████████████████████████████| 2.1MB 5.9MB/s 
     |████████████████████████████████| 870kB 36.1MB/s 
     |████████████████████████████████| 3.3MB 36.1MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.44-cp37-none-any.whl size=886084 sha256=ac63686ae7b289e927e2cc15777d24c0120dfd8678def8ea26231eaf4b083a61
  Stored in directory: /root/.cache/pip/wheels/3e/fb/c0/13ab4d63d537658f448366744654323077c4d90069b6512f3c
Successfully built sacremoses


In [6]:
!pip install sentencepiece

     |████████████████████████████████| 1.2MB 3.7MB/s 


In [7]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences

from transformers import *
import numpy as np
import argparse
from tqdm import tqdm 
import random
import transformers
import pandas as pd
from collections import Counter
from transformers import get_linear_schedule_with_warmup
import sys
sys.argv=['']
del sys

In [8]:
df = pd.read_csv('combine.csv')
X = df['sentence'].values
Y = df['label'].values
index = np.arange(X.shape[0])
np.random.shuffle(index)
X = X[index]
Y = Y[index]

In [9]:
from collections import Counter

In [10]:
Counter(Y)

Counter({0: 15775, 1: 515, 2: 16822})

In [11]:

!pip install imbalanced-learn

In [12]:
import imblearn

/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [13]:
from imblearn.over_sampling import SVMSMOTE

In [14]:
import numpy as np
# import unidecode
import pandas as pd
import string
import re
import os
import shutil
import pickle 

In [15]:
def transform(x):
  return [s.lower() for s in x]
# def remove_tone(x):
#   return [unidecode.unidecode(s) for s in x] 
def remove_duplicate(x):
  result = []
  for s in x:
      s = re.sub(r'([a-z])\1+', lambda m: m.group(1), s, flags=re.IGNORECASE)
      s = re.sub(r'([a-z][a-z])\1+', lambda m: m.group(1), s, flags=re.IGNORECASE)
      result.append(s)
  return result

In [16]:
X = transform(X)
X = remove_duplicate(X)

In [17]:
parser = argparse.ArgumentParser()
parser.add_argument('--epochs', type=int, default=10)
parser.add_argument('--batch_size', type=int, default=8)
parser.add_argument('--maxlen', type=int, default=120)
parser.add_argument('--seed', type=int, default=42)
parser.add_argument('--lr', type=float, default=2e-5)
args = parser.parse_args()

random.seed(args.seed)
np.random.seed(args.seed)
torch.manual_seed(args.seed)
torch.cuda.manual_seed_all(args.seed)

MAX_LEN = args.maxlen

In [18]:
MAX_LEN

120

In [19]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-uncased', do_lower_case=True)

In [20]:
input_ids = []
filtered_labels = []
for i, sent in tqdm(enumerate(X)):
      encoded_sent = tokenizer.encode(sent,add_special_tokens = True)
      input_ids.append(encoded_sent)
      filtered_labels.append(Y[i])
inputs = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", 
                                    value=0, truncating="post", padding="post")                                                     

1827it [00:02, 722.07it/s]Token indices sequence length is longer than the specified maximum sequence length for this model (645 > 512). Running this sequence through the model will result in indexing errors
33112it [00:45, 727.31it/s]


In [21]:
inputs

array([[  101, 17840, 11439, ...,     0,     0,     0],
       [  101, 12277, 11928, ...,     0,     0,     0],
       [  101, 13718, 30066, ...,     0,     0,     0],
       ...,
       [  101, 10492, 10910, ...,     0,     0,     0],
       [  101, 30066, 10910, ...,     0,     0,     0],
       [  101, 11042, 40677, ...,     0,     0,     0]])

In [22]:
inputs.shape

(33112, 120)

In [23]:
len(filtered_labels)

33112

In [24]:
print(filtered_labels)

[2, 0, 2, 2, 0, 2, 0, 2, 0, 0, 2, 2, 0, 2, 2, 0, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 2, 2, 0, 2, 0, 0, 0, 2, 0, 2, 0, 0, 2, 2, 0, 2, 2, 0, 0, 0, 2, 0, 0, 0, 2, 0, 2, 2, 0, 0, 2, 2, 0, 0, 2, 0, 0, 0, 2, 2, 2, 0, 0, 0, 0, 2, 0, 2, 0, 2, 0, 2, 0, 0, 2, 0, 0, 0, 2, 2, 0, 0, 0, 0, 2, 2, 2, 2, 2, 0, 0, 0, 0, 2, 0, 2, 2, 0, 0, 0, 2, 0, 2, 2, 2, 2, 0, 2, 0, 2, 0, 0, 2, 0, 2, 0, 2, 0, 0, 0, 0, 0, 0, 2, 0, 0, 2, 0, 2, 0, 2, 2, 2, 0, 0, 0, 2, 2, 0, 2, 2, 2, 2, 0, 0, 2, 0, 0, 0, 2, 0, 2, 2, 0, 0, 0, 2, 1, 0, 2, 2, 0, 2, 0, 2, 0, 1, 0, 2, 0, 2, 0, 2, 2, 2, 2, 0, 0, 2, 2, 0, 0, 0, 2, 2, 0, 2, 2, 0, 2, 2, 2, 2, 0, 2, 0, 2, 2, 2, 2, 2, 2, 0, 2, 1, 0, 0, 0, 2, 0, 2, 0, 2, 0, 0, 2, 2, 2, 2, 2, 2, 0, 2, 2, 0, 1, 2, 0, 0, 0, 2, 2, 2, 2, 0, 0, 2, 2, 0, 0, 0, 2, 0, 0, 2, 2, 2, 1, 0, 2, 0, 2, 2, 1, 2, 2, 0, 0, 2, 2, 2, 2, 0, 0, 2, 0, 2, 0, 0, 2, 0, 2, 2, 0, 2, 0, 0, 0, 0, 0, 2, 0, 2, 2, 0, 2, 2, 2, 0, 2, 0, 2, 0, 0, 0, 2, 2, 2, 2, 2, 2, 0, 2, 0, 0, 0, 0, 0, 2, 0, 0, 1, 2, 2, 2, 0, 0, 0, 0, 0, 2, 0, 2, 2, 2, 2, 2, 

In [25]:
oversample = SVMSMOTE()
inputs, filtered_labels = oversample.fit_resample(inputs, filtered_labels)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarni

In [26]:

Counter(filtered_labels)

Counter({0: 16822, 1: 16822, 2: 16822})

In [27]:
inputs.shape

(50466, 120)

In [28]:
len(filtered_labels)

50466

In [29]:
masks = []

for sent in inputs:
    att_mask = [int(token_id > 0) for token_id in sent]
    masks.append(att_mask)

In [30]:
inputs = torch.tensor(inputs)
labels = torch.tensor(filtered_labels)
masks = torch.tensor(masks)

In [31]:
index = np.arange(inputs.shape[0])
np.random.shuffle(index)
inputs = inputs[index]
labels = labels[index]
masks = masks[index]

In [32]:
labels 

tensor([2, 1, 2,  ..., 2, 0, 0])

In [33]:
np.unique(labels)

array([0, 1, 2])

In [34]:
train_inputs = inputs[:-10000]
train_labels = labels[:-10000]
train_masks = masks[:-10000]
val_inputs = inputs[-10000:]
val_labels = labels[-10000:]
val_masks= masks[-10000:]

In [35]:


train_labels 

tensor([2, 1, 2,  ..., 2, 0, 2])

In [36]:
len(train_labels[train_labels==2])

13417

In [37]:
len(val_labels[val_labels==1])

3261

In [38]:
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=args.batch_size)
test_data = TensorDataset(val_inputs, val_masks, val_labels)
test_sampler = RandomSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=args.batch_size)

In [39]:
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [40]:
def get_baseline_acc(model, validation_dataloader, device):
    model.eval()

    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    for batch in validation_dataloader:
        batch = tuple(t.to(device) for t in batch)
        
        b_input_ids, b_input_mask, b_labels = batch
        
        with torch.no_grad():        
            outputs = model(b_input_ids, 
                    token_type_ids=None, 
                    attention_mask=b_input_mask, 
                    labels=b_labels)
        
        logits = outputs[1]

        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1

    eval_accuracy = eval_accuracy/nb_eval_steps
    return eval_accuracy

In [41]:
if torch.cuda.is_available():    
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

In [42]:
device

device(type='cuda')

In [43]:
model = BertForSequenceClassification.from_pretrained("bert-base-multilingual-uncased", num_labels=3)

Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model 

In [49]:
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(105879, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [44]:
optimizer = AdamW(model.parameters(),lr = args.lr)
total_steps = len(train_dataloader) * args.epochs
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps = int(0.1*total_steps),
                                            #warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)
                                            #t_total = total_steps)

In [45]:
!nvidia-smi

Wed Apr 14 14:44:07 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.67       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8    26W / 149W |      3MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [46]:
device

device(type='cuda')

In [47]:
print(train_dataloader)

In [50]:
for epoch_i in range(args.epochs):
    total_loss = 0
    model.train()
        
    # For each batch of training data...
    for step, batch in tqdm(enumerate(train_dataloader)):
        model.train()
        
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        model.zero_grad()      
        outputs = model(b_input_ids, 
                    token_type_ids=None, 
                    attention_mask=b_input_mask, 
                    labels=b_labels)
        
        loss = outputs[0]
        loss.backward()

        total_loss += loss.item()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        model.zero_grad()

    # Calculate the average loss over the training data.
    avg_train_loss = total_loss / len(train_dataloader)            
    

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))

    test_accuracy = get_baseline_acc(model, test_dataloader, device)
    print(" Test acc {}".format( test_accuracy))

print("")
print("Training complete!")

0it [00:00, ?it/s]


RuntimeError: ignored